# **OptimalpH-Colab**: Predict Optimal pH for Hydrolase Activity in Google Colab

## Introduction

OptimalpH is a tool designed to predict the optimal pH for hydrolase enzymes activity using machine learning models. This Google Colab notebook allows for batch processing of protein sequences to predict the pH range where hydrolases exhibit maximum catalytic activity.

The notebook utilizes ESM2-650M protein language model embeddings developed by Meta AI Research to train machine learning models including KNN, XGBoost, and K-mers.


---
### Author Information

This notebook provides a user-friendly interface for predicting a collection of metrics that aid in the expression, purification, and sceeening of hydrolases created by <b><font color='Gold'>**Logan Hessefort**</font></b> ([LinkedIn](https://www.linkedin.com/in/logan-hessefort/)).

This notebook was created as part of a <b><font color='green'>US Department of Energy SCGSR Fellowship</font></b> ([details](https://science.osti.gov/wdts/scgsr)) at the National Renewable Energy Laboratory with additional support from the <b><font color='DodgerBlue'>US National Science Foundation</font></b> ([grant](https://www.nsf.gov/awardsearch/showAward?AWD_ID=2132183&HistoricalAwards=false)).


---


### Training Data

The models were trained using approximately **2,840 data points from the BRENDA** enzyme database and **169,517 data points from the Mean Growth pH dataset**. The data were processed through clustering techniques such as DBSCAN and CD-HIT, followed by hyperparameter tuning and evaluation on various splits to optimize prediction accuracy.


---


### Citation and Acknowledgment
This tool is based on the research presented in the paper **"Approaching Optimal pH Enzyme Prediction with Large Language Models"** by Mark Zaretckii, Pavel Buslaev, Igor Kozlovskii, Alexander Morozov, and Petr Popov. The original paper can be accessed [here](https://doi.org/10.1021/acssynbio.4c00465). Thanks to the authors for their excellent work.



---



## How to Use This Notebook

1. Hit Runtime > Run all to:
   - Set up the necessary environment
   - Clone the OptimalpH repository
   - Install required dependencies
   - Download pre-trained models from Hugging Face

2. Prepare your input CSV file (see the next section for details).

3. Upload your CSV file when prompted.

4. Select the desired model type

5. The notebook will process each sequence and output a consolidated CSV file with the results.

## Input CSV Structure

Your input CSV should contain the following columns:

- `ID`: A unique identifier for each protein sequence
- `Sequence`: The amino acid sequence of the protein

Your CSV should resemble this format:

| ID       | Sequence       |
|:--------:|:--------------:|
| Protein1 | MVKPKLFYV...    |
| Protein2 | MGSRHYPLG...    |
| ...      | ...             |
| ProteinN | MATLYPLET...    |

<font color="cyan">**Note:** Ensure all sequences are valid amino acid sequences using single-letter codes.</font>

## Output

The script will generate a comprehensive CSV file containing the following columns:

- ID
- Sequence
- Molecular Weight
- Oxidized Cystine Extinction Coefficient (M-1 cm-1, @ 280 nm measured in water)
- Isoelectric Point
- Predicted Optimal pH
- Lysine Count
- Arginine Count
- Cysteine Count
- Instability Index
- Stability Prediction (Stable, Moderately Stable, Unstable)
- Hydrophobicity Score



---



# Key Protein Properties Explained

### GRAVY (Grand Average of Hydropathy) Score
- Measures overall protein hydrophobicity
- Range: Typically -2 to +2
- Interpretation:
  - Positive scores: More hydrophobic, likely membrane-associated
  - Negative scores: More hydrophilic, likely soluble
  - Higher scores suggest decreased water solubility

### Isoelectric Point (pI)
- pH at which a protein carries no net electrical charge
- Range: Typically 3 to 12
- Significance:
  - Affects protein solubility and stability
  - Proteins are least soluble at their pI
  - Important for protein purification and behavior in different pH environments

### Instability Index
- Estimates protein stability *in vitro*
- Range: No fixed range, but generally:
  - ≤ 40: Predicted as stable
  - > 40: Predicted as unstable
- Based on dipeptide composition
- Higher values suggest decreased stability in test tube conditions

<font color="pink">These properties are simplified measures but they can provide some valuable insights into protein behavior, helping to predict stability, solubility, and potential cellular localization.</font>


---


For any issues or questions, please refer to the [OptimalpH GitHub repository](https://github.com/Loganz97/optimalpH_colab).

In [ ]:
# @title Installing the necessary dependencies
# Install required packages quietly
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122
%pip install -q pandas numpy scikit-learn xgboost biopython fire huggingface_hub fair-esm==2.0.0 fairscale==0.4.13 fsspec

# Clone the OptimalpH repository quietly
!git clone -q https://github.com/Loganz97/optimalpH_colab.git > /dev/null 2>&1
%cd /content/
%cd /content/optimalpH_colab

# Add this line to ensure the correct Python path
import sys
sys.path.append('/content/optimalpH_colab')

# Necessary imports
import pandas as pd
from google.colab import files
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from io import StringIO
import subprocess
import os
from huggingface_hub import hf_hub_download
import torch

# Check CUDA availability and print only the device name if available
if torch.cuda.is_available():
    torch.cuda.set_device(0)
    print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")

# Download weights from Hugging Face quietly
model_id = "Loganz97/optimalpH"
model_files = ["model_kmers", "model_knn", "model_xgboost"]
local_weights_paths = {}

for model_file in model_files:
    local_path = hf_hub_download(repo_id=model_id, filename=f"weights/{model_file}")
    local_weights_paths[model_file] = local_path

# **Model Selection and Performance Overview**

- **XGBoost (Most Accurate)**:
  - **Accuracy**: This model delivers the highest accuracy with a mean absolute error (MAE) of approximately 0.6.
  - **Pearson Correlation**: Shows a strong correlation with actual pH values, typically around 0.85, making it the most reliable option for precise predictions.

- **K-Nearest Neighbors (KNN)**:
  - **Accuracy**: Offers robust performance with an MAE of approximately 0.7.
  - **Pearson Correlation**: Displays a solid correlation, generally around 0.80, ensuring reliable predictions across diverse datasets.

- **K-mers**:
  - **Accuracy**: Has a higher error rate compared to XGBoost and KNN, with an MAE around 0.9.
  - **Pearson Correlation**: Exhibits a moderate correlation, typically around 0.75, making it useful for broader trend analysis rather than precise predictions.

-  <font color="cyan">**Note**: These models have been primarily trained and optimized for hydrolases. Performance may vary for other enzyme classes or protein types.</font>

In [ ]:
# @title OptimalpH Model Selection - Upload your CSV file below ⬇️

# Model Selection
model_selection = "XGBoost (Most Accurate)" #@param ["XGBoost (Most Accurate)", "KNN (Also Performs Well)", "K-mers"]

# Map the user-friendly model names to the actual model file names
model_map = {
    "XGBoost (Most Accurate)": "model_xgboost",
    "KNN (Also Performs Well)": "model_knn",
    "K-mers": "model_kmers"
}

selected_model = model_map[model_selection]

def calculate_protein_properties(sequence):
    try:
        if not isinstance(sequence, str):
            raise ValueError(f"Invalid sequence type: {type(sequence)}. Expected string.")

        sequence = ''.join(sequence.split()).upper()
        analysis = ProteinAnalysis(sequence)

        molecular_weight = analysis.molecular_weight()
        extinction_coefficient = analysis.molar_extinction_coefficient()[0]  # Oxidized
        isoelectric_point = analysis.isoelectric_point()
        instability_index = analysis.instability_index()

        if instability_index <= 30:
            instability_gauge = "Stable"
        elif 30 < instability_index <= 40:
            instability_gauge = "Moderately Stable"
        else:
            instability_gauge = "Unstable"

        lysine_count = sequence.count('K')
        arginine_count = sequence.count('R')
        cysteine_count = sequence.count('C')
        hydrophobicity = analysis.gravy()

        return (molecular_weight, extinction_coefficient, isoelectric_point, lysine_count,
                arginine_count, cysteine_count, instability_index, instability_gauge,
                hydrophobicity)
    except Exception as e:
        print(f"Error processing sequence: {e}")
        return tuple([None] * 9)

# Upload CSV file
print("Please upload your CSV file containing protein sequences.")
uploaded = files.upload()

# Read the uploaded CSV file
file_name = list(uploaded.keys())[0]
df = pd.read_csv(StringIO(uploaded[file_name].decode('utf-8')))

print(f"Uploaded file: {file_name}")
print(f"Number of sequences: {len(df)}")

# Ensure we have 'ID' and 'Sequence' columns
if 'ID' not in df.columns or 'Sequence' not in df.columns:
    raise ValueError("The CSV file must contain 'ID' and 'Sequence' columns.")

# Calculate protein properties for each sequence
results = df['Sequence'].apply(calculate_protein_properties)

# Add new columns to the dataframe in the specified order
new_columns = ['Molecular Weight', 'Oxidized Extinction Coefficient', 'Isoelectric Point',
               'Lysine Count', 'Arginine Count', 'Cysteine Count', 'Instability Index',
               'Instability Gauge', 'Hydrophobicity']

for i, col in enumerate(new_columns):
    df[col] = [result[i] for result in results]

# Run the OptimalpH model
input_file = 'input_sequences.csv'
df[['ID', 'Sequence']].to_csv(input_file, index=False)
output_file = 'optimalpH_results.csv'

command = f"python3 code/predict.py --input_csv {input_file} --id_col ID --seq_col Sequence --model_fname {local_weights_paths[selected_model]} --output_csv {output_file}"

try:
    result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
except subprocess.CalledProcessError as e:
    print(f"An error occurred while running the OptimalpH model. Return code: {e.returncode}")
    sys.exit(1)

# Check if the output file exists
if os.path.exists(output_file):
    optimalpH_results = pd.read_csv(output_file)
    optimalpH_results = optimalpH_results.rename(columns={'Predicted_pH': 'predict_optimal_pH'})
    df = pd.merge(df, optimalpH_results[['ID', 'predict_optimal_pH']], on='ID', how='left')
else:
    print(f"Warning: OptimalpH output file '{output_file}' not found.")
    sys.exit(1)

# Reorder the columns
final_columns = ['ID', 'Sequence', 'Molecular Weight', 'Oxidized Extinction Coefficient',
                 'Isoelectric Point', 'predict_optimal_pH', 'Lysine Count', 'Arginine Count',
                 'Cysteine Count', 'Instability Index', 'Instability Gauge', 'Hydrophobicity']

df = df[final_columns]

# Save the final comprehensive results to a CSV file
final_output_file = 'comprehensive_protein_analysis_results.csv'
df.to_csv(final_output_file, index=False)

print(f"Analysis complete. Results saved as '{final_output_file}'.")
print("The output CSV will be automatically downloaded.")

# Download the result file
files.download(final_output_file)